In [69]:
import findspark

findspark.init()

In [70]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.master('yarn').appName('als').getOrCreate()
sc= spark.sparkContext

In [71]:
from pyspark.sql.types import StructType,StructField,IntegerType,StringType,DoubleType

colNames = ['movieId','title','genres']
movies_schema = StructType()


In [72]:
for name in colNames:
    if name == 'movieId':
        movies_schema.add(StructField(name,IntegerType(),True))
    else:
        movies_schema.add(StructField(name,StringType(),True))
        


In [73]:
movies = spark.read.csv('s3://ssds00/ml-latest/movies.csv',header=True,schema=movies_schema)

In [74]:
movies.show(5,truncate=False)

+-------+----------------------------------+-------------------------------------------+
|movieId|title                             |genres                                     |
+-------+----------------------------------+-------------------------------------------+
|1      |Toy Story (1995)                  |Adventure|Animation|Children|Comedy|Fantasy|
|2      |Jumanji (1995)                    |Adventure|Children|Fantasy                 |
|3      |Grumpier Old Men (1995)           |Comedy|Romance                             |
|4      |Waiting to Exhale (1995)          |Comedy|Drama|Romance                       |
|5      |Father of the Bride Part II (1995)|Comedy                                     |
+-------+----------------------------------+-------------------------------------------+
only showing top 5 rows



In [75]:

ratingColNames = ['userId','movieId','rating','timestamp']
rating_schema = StructType()
for r_col in ratingColNames:
    if r_col == 'rating':
        rating_schema.add(StructField(r_col,DoubleType(),True))
    else:
        rating_schema.add(StructField(r_col,IntegerType(),True))
        

In [76]:
rating_schema

StructType(List(StructField(userId,IntegerType,true),StructField(movieId,IntegerType,true),StructField(rating,DoubleType,true),StructField(timestamp,IntegerType,true)))

In [77]:

ratings = spark.read.csv('s3://ssds00/ml-latest/ratings.csv',header=True,schema=rating_schema)

In [78]:
ratings.show(5,truncate=False)

+------+-------+------+----------+
|userId|movieId|rating|timestamp |
+------+-------+------+----------+
|1     |307    |3.5   |1256677221|
|1     |481    |3.5   |1256677456|
|1     |1091   |1.5   |1256677471|
|1     |1257   |4.5   |1256677460|
|1     |1449   |4.5   |1256677264|
+------+-------+------+----------+
only showing top 5 rows



In [23]:
from pyspark.ml.recommendation import ALS

movie_rating = movies.join(ratings,on='movieId',how='inner')


In [24]:
movie_rating.show(3)

+-------+--------------------+--------------+------+------+----------+
|movieId|               title|        genres|userId|rating| timestamp|
+-------+--------------------+--------------+------+------+----------+
|    307|Three Colors: Blu...|         Drama|     1|   3.5|1256677221|
|    481|   Kalifornia (1993)|Drama|Thriller|     1|   3.5|1256677456|
|   1091|Weekend at Bernie...|        Comedy|     1|   1.5|1256677471|
+-------+--------------------+--------------+------+------+----------+
only showing top 3 rows



In [79]:
train,test = ratings.randomSplit([0.8,0.2])

In [80]:
als = ALS(maxIter=10,regParam=1.0,userCol='userId',itemCol='movieId',ratingCol='rating',coldStartStrategy='drop')
model = als.fit(train)

In [81]:
predictions = model.transform(test)

In [82]:
predictions.show(5,truncate=False)

+------+-------+------+---------+----------+
|userId|movieId|rating|timestamp|prediction|
+------+-------+------+---------+----------+
|211963|148    |3.0   |903702432|1.5759593 |
|75370 |148    |5.0   |829760897|2.2315795 |
|144004|148    |5.0   |828975154|2.5027206 |
|168206|148    |4.0   |939071406|2.1441715 |
|241107|148    |2.0   |879040382|1.8247002 |
+------+-------+------+---------+----------+
only showing top 5 rows



In [29]:
from pyspark.ml.evaluation import RegressionEvaluator
evaluator = RegressionEvaluator(metricName='rmse',labelCol='rating',predictionCol='prediction')
rmse = evaluator.evaluate(predictions)
print('root-mean-square error = ' + str(rmse))

root-mean-square error = 1.3217849706355098


In [30]:
userRecs = model.recommendForAllItems(10)

In [40]:
userRecs.show(n=2,truncate=False)

+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|movieId|recommendations                                                                                                                                                                                             |
+-------+------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+
|148    |[[140712, 3.2697585], [247240, 3.2686515], [10538, 3.2669492], [27174, 3.266422], [167730, 3.2628908], [88857, 3.2628908], [99142, 3.2628908], [278033, 3.2606065], [37564, 3.260213], [107323, 3.2600183]] |
|463    |[[140712, 3.2263522], [247240, 3.2252598], [10538, 3.2235808], [27174, 3.2230752], [99142, 3.2195764], [88857, 3.2195764], [167730,

In [84]:
import pandas as pd
movies_pd = movies.toPandas().set_index('movieId')
targetUserID = 1677
userWatched = ratings.where(ratings.userId == targetUserID)
userWatchedPD = userWatched.toPandas()
userWatchedPD['title']=userWatchedPD['movieId'].apply(lambda x : movies_pd.loc[x].title)
userWatchedPD

,userId,movieId,rating,timestamp,title
0,1677,318,5.0,1453580813,"Shawshank Redemption, The (1994)"
1,1677,527,3.0,1453580818,Schindler's List (1993)
2,1677,593,5.0,1453580893,"Silence of the Lambs, The (1991)"
3,1677,1240,3.5,1453581254,"Terminator, The (1984)"
4,1677,1704,5.0,1496670433,Good Will Hunting (1997)
5,1677,2028,3.5,1453580912,Saving Private Ryan (1998)
6,1677,2571,5.0,1453580801,"Matrix, The (1999)"
7,1677,3578,3.5,1453581191,Gladiator (2000)
8,1677,4306,2.0,1453581281,Shrek (2001)
9,1677,4995,5.0,1470606315,"Beautiful Mind, A (2001)"


In [37]:
user = userWatched.distinct()
user.show(3)


+------+-------+------+----------+
|userId|movieId|rating| timestamp|
+------+-------+------+----------+
|  1677|   4995|   5.0|1470606315|
|  1677|  54286|   4.0|1453581221|
|  1677|   3578|   3.5|1453581191|
+------+-------+------+----------+
only showing top 3 rows



In [39]:
userSubsetRecs = model.recommendForUserSubset(user,10).toPandas()
userSubsetRecs['title'] = userSubsetRecs['recommendations'].apply(lambda x : [movies_pd.loc[i[0]].title for i in x])
for item in userSubsetRecs.title:
    print(item)

['Baseline (2010)', 'Space Dive (2012)', 'The Thorn (1971)', 'Caedes (2015)', '3-D Sex and Zen: Extreme Ecstasy (2011)', 'Amori che non sanno stare al mondo (2017)', 'Ciutat morta (2014)', 'Pale (2016)', 'Acı Aşk (2009)', 'I Can No Longer Hear the Guitar (1991)']


In [85]:
userSubsetRecs

,userId,recommendations,title
0,1677,"[(138172, 4.326651573181152), (189503, 4.32411...","[Baseline (2010), Space Dive (2012), The Thorn..."


In [86]:
movies = ratings.select(als.getItemCol()).distinct().limit(3)
movieSubSetRecs = model.recommendForItemSubset(movies,10)
movieSubSetRecs.toPandas()


,movieId,recommendations
0,48780,"[(140712, 4.530224323272705), (247240, 4.52820..."
1,69481,"[(140712, 4.185409069061279), (247240, 4.18354..."
2,110433,"[(140712, 3.2430856227874756), (247240, 3.2416..."


In [89]:
import pyspark.sql.functions as sf
from pyspark.sql.functions import col
m_pd = movieSubSetRecs.toPandas()
usertb = movies.select(col('userId'))
# m_pd['recommendations'].apply(lambda x : )

AnalysisException: "cannot resolve '`userId`' given input columns: [movieId];;\n'Project ['userId]\n+- GlobalLimit 3\n   +- LocalLimit 3\n      +- Deduplicate [movieId#613]\n         +- Project [movieId#613]\n            +- Relation[userId#612,movieId#613,rating#614,timestamp#615] csv\n"

In [42]:
als.getItemCol()

'movieId'

In [44]:
movies.show()

+-------+
|movieId|
+-------+
|  48780|
|  69481|
| 110433|
+-------+



In [51]:
userSubsetRecs.head(3)

movieRecs = model.recommendForAllItems(10)

,userId,recommendations,title
0,1677,"[(138172, 4.326651573181152), (189503, 4.32411...","[Baseline (2010), Space Dive (2012), The Thorn..."


In [53]:
moviesrecs_pd = movieRecs.toPandas()
moviesrecs_pd.head()

,movieId,recommendations
0,148,"[(140712, 3.269758462905884), (247240, 3.26865..."
1,463,"[(140712, 3.2263522148132324), (247240, 3.2252..."
2,471,"[(140712, 4.129149913787842), (247240, 4.12775..."
3,496,"[(140712, 3.761198043823242), (247240, 3.75992..."
4,833,"[(140712, 3.152521848678589), (247240, 3.15145..."


In [93]:
from pyspark.ml.feature import StringIndexer

from pyspark.sql.types import StringType,StructField,StructType,DoubleType,IntegerType

base_path = 's3://ssds00/'
triplet_path = base_path+'triplet.txt'
song_path = base_path +'song.csv'
print(song_path)
songs = spark.read.csv('s3://ssds00/triplet.txt')

s3://ssds00/song.csv


Py4JJavaError: An error occurred while calling o1233.csv.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: 655B5AF455061BC7; S3 Extended Request ID: DT9V7h0RTY/vH/Qsufyp2fxs8/oO0hj8KyOBwKEgsoFDC3Nylb29JxgfENQCQb1HMxargjUiaW0=), S3 Extended Request ID: DT9V7h0RTY/vH/Qsufyp2fxs8/oO0hj8KyOBwKEgsoFDC3Nylb29JxgfENQCQb1HMxargjUiaW0=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:286)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:781)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:347)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scal

In [96]:
songs = spark.read.csv('s3://ssds00/song.csv')

Py4JJavaError: An error occurred while calling o1322.csv.
: java.io.IOException: com.amazon.ws.emr.hadoop.fs.shaded.com.amazonaws.services.s3.model.AmazonS3Exception: Access Denied (Service: Amazon S3; Status Code: 403; Error Code: AccessDenied; Request ID: A913282A5CDA00D9; S3 Extended Request ID: WL3sgCYdC73KtqAlFSPnUS46zBJCY7b9VvHV2IQZZP/dCqaoVnccUlqK9kjCK6EeOZyze9JJIMM=), S3 Extended Request ID: WL3sgCYdC73KtqAlFSPnUS46zBJCY7b9VvHV2IQZZP/dCqaoVnccUlqK9kjCK6EeOZyze9JJIMM=
	at com.amazon.ws.emr.hadoop.fs.s3n.Jets3tNativeFileSystemStore.list(Jets3tNativeFileSystemStore.java:286)
	at com.amazon.ws.emr.hadoop.fs.s3n.S3NativeFileSystem.getFileStatus(S3NativeFileSystem.java:781)
	at org.apache.hadoop.fs.FileSystem.exists(FileSystem.java:1440)
	at com.amazon.ws.emr.hadoop.fs.EmrFileSystem.exists(EmrFileSystem.java:347)
	at org.apache.spark.sql.execution.datasources.DataSource$$anonfun$org$apache$spark$sql$execution$datasources$DataSource$$checkAndGlobPathIfNecessary$1.apply(DataSource.scal

In [60]:
songs.head(3)

[Row(_c0='movieId', _c1='title', _c2='genres'),
 Row(_c0='1', _c1='Toy Story (1995)', _c2='Adventure|Animation|Children|Comedy|Fantasy'),
 Row(_c0='2', _c1='Jumanji (1995)', _c2='Adventure|Children|Fantasy')]